# MCMC3.5: Jackknife Resampling

To estimate the error of observables accurately, it is recommended to use the Jackknife resampling method.

## Binning

Previously, we faced a large errorbar problem at low temperature in this program:

In [1]:
using ResumableFunctions
using SparseArrays
using LinearAlgebra
const Jx = 1 / 3 # oppposite sign to Motome's
const Jy = 1 / 3
const Jz = 1 / 3
function Metropolis(βF::Float64, βFnew::Float64)::Bool
    βF - βFnew > log(1.0 - rand())
end
function openhoneycomb(Lx::Int64, Ly::Int64)::Tuple
    N = 2Lx * Ly
    nnx = zip(1 : 2 : (N - 1), 2 : 2 : N)
    nny = Iterators.flatten((zip((1 + 2i) : 2Lx : (2Lx * (Ly - 1)  + 1 + 2i), 2i : 2Lx : (2Lx * (Ly - 1)  + 2i)) for i in 1 : (Lx - 1)))
    nnz = zip(1 : 2 : (N - 1), Iterators.flatten(((2Lx + 2) : 2 : N, 2 : 2 : 2Lx)))
    plaquette = Iterators.flatten(zip((Lx * (i - 1) + 1) : (Lx * (i - 1) + Lx - 1), (Lx * (i - 1) + 2) : (Lx * (i - 1) + Lx)) for i in 1 : Ly)
    N, nnx, nny, nnz, plaquette
end
@resumable function measurementflux(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Float64
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 2.0im * J
        h[nn[2], nn[1]] = -2.0im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    hdense = Array(h)
    plaq = collect(plaquette)
    Np = length(plaq)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            ev = eigvals(Hermitian(hdense))
            positiveev = Iterators.drop(ev, N >> 1)
            βFnew = -sum((log(2.0 * cosh(β * ϵ / 2.0)) for ϵ in positiveev))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        @yield sum((η[k] * η[l] for (k, l) in plaq)) / Np
    end
end

measurementflux (generic function with 1 method)

One reason is the discreteness of the returned value `flux`.

In [2]:
mcstep = Iterators.drop(measurementflux(Metropolis, openhoneycomb, 25.0, 4, 4), 10000)
foreach(println, Iterators.take(mcstep, 10))

0.3333333333333333
0.5
0.0
0.3333333333333333
0.8333333333333334
1.0
0.5
0.5
0.0
0.16666666666666666


We have to flatten these quantized values by binning.

In [3]:
using Statistics
Nsample = 10000
Nbin = 10
Nbinsize = Nsample ÷ Nbin
iter = Iterators.partition(Iterators.take(mcstep, Nsample), Nbinsize)
bin = collect(map(mean, iter))

10-element Array{Float64,1}:
 0.32800000000000007
 0.31766666666666665
 0.3163333333333333 
 0.2968333333333334 
 0.29716666666666663
 0.3056666666666667 
 0.29399999999999993
 0.2986666666666666 
 0.3083333333333333 
 0.2978333333333334 

Now it works!

In [4]:
m = mean(bin)
s = stdm(bin, m) / sqrt(length(bin))
println("$m ± $s")

0.30605000000000004 ± 0.003588480368906137


`Nbinsize` has to be determined based on the autocorrelation. In order to reduce `Nbin` to get a more acculate result at low temperature, we need to implement some global updating algorithm.

## Delete-1 jackknife resampling

Delete-1 jackknife resampling is simply implemented in https://github.com/ararslan/Jackknife.jl. However, the function is limited, so I will newly define functions for the jackknife resampling.

In [5]:
function leaveoneout(before::Function, after::Function, v::AbstractVector)
    ind = eachindex(v)
    map(i -> after(mean(map(before, view(v, filter(!isequal(i), ind))))), ind)
end
meanJ(b::Function, a::Function, v::AbstractVector) = mean(leaveoneout(b, a, v))
stdmJ(b::Function, a::Function, v::AbstractVector, m) = stdm(leaveoneout(b, a, v), m, corrected = false) * sqrt(length(v) - 1)
stdJ(b::Function, a::Function, v::AbstractVector) = stdmJ(b, a, v, meanJ(b, a, v))

stdJ (generic function with 1 method)

The functions are based on Statistics.jl and Jackknife.jl, so please see their reference to know how it works. I again use `measurementEf` as a demonstration.

In [6]:
@resumable function measurementEf(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Vector{Float64}
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 2.0im * J
        h[nn[2], nn[1]] = -2.0im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    β₂ = β * 0.5
    hdense = Array(h)
    ev = zeros(Float64, N)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            evnew = eigvals(Hermitian(hdense))
            βFnew = -sum(@. log(exp(β₂ * evnew[(N >> 1 + 1) : end]) + exp(-β₂ * evnew[(N >> 1 + 1) : end])))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
                ev .= evnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        Ef = -sum(@. ev[(N >> 1 + 1) : end] * tanh(β₂ * ev[(N >> 1 + 1) : end] )) * 0.5
        ∂Ef∂β = -sum(@. (ev[(N >> 1 + 1) : end] ^ 2) * (sech(β₂ * ev[(N >> 1 + 1) : end]) ^ 2)) * 0.25
        @yield [Ef, ∂Ef∂β]
    end
end

measurementEf (generic function with 1 method)

It is ok to first assume the bin size to be 1.

In [7]:
const β = 10.0
mcstep2 = Iterators.drop(measurementEf(Metropolis, openhoneycomb, β, 4, 4), 10000)
iter2 = Iterators.take(mcstep2, Nsample)
data = collect(iter2)

10000-element Array{Array{Float64,1},1}:
 [-7.69719, -0.0124383] 
 [-7.77122, -0.00996081]
 [-7.64995, -0.00720097]
 [-7.55941, -0.0103376] 
 [-7.74858, -0.0124451] 
 [-7.77122, -0.00996081]
 [-7.70063, -0.0120469] 
 [-7.65228, -0.0072398] 
 [-7.59156, -0.00866315]
 [-7.56603, -0.00975472]
 [-7.69172, -0.0118235] 
 [-7.64676, -0.0072279] 
 [-7.75187, -0.00914975]
 ⋮                      
 [-7.56369, -0.00772785]
 [-7.59986, -0.00752412]
 [-7.64608, -0.00726712]
 [-7.74996, -0.0101327] 
 [-7.60048, -0.00845927]
 [-7.67102, -0.0117057] 
 [-7.66072, -0.0133364] 
 [-7.70367, -0.0128367] 
 [-7.76389, -0.0103935] 
 [-7.57655, -0.00905835]
 [-7.62655, -0.0145969] 
 [-7.73337, -0.0103165] 

By setting `after` = `before` = `identity`, `meanJ` and `stdmJ `work in the same way as `mean` and `stdm`, respectively.

In [8]:
m2 = meanJ(identity, identity, data)
s2 = stdmJ(identity, identity, data, m2)
println("Ef = $(m2[1]) ± $(s2[1]), ∂Ef∂β = $(m2[2]) ± $(s2[2])")

Ef = -7.690819599997395 ± 0.0006711072111108698, ∂Ef∂β = -0.010791993504182517 ± 2.5928539374598384e-5


This agrees with the standard estimation method for the error bars.

In [9]:
std(data) / sqrt(length(data))

2-element Array{Float64,1}:
 0.0006711072111121578
 2.592853937460015e-5 

For such mean values, the jackknife resampling is apparently overkill. However, to estimate the error for the values like the specific heat, the jackknife resampling is very effective.

In [10]:
TTCv(v::Vector{Float64}) = [v[1] ^ 2 - v[2], v[1]]
function Cv(β::Float64)::Function
    β² = β ^ 2
    meanTTCv::Vector{Float64} -> β² * (meanTTCv[1] - meanTTCv[2] ^ 2)
end
m3 = meanJ(TTCv, Cv(β), data)
s3 = stdmJ(TTCv, Cv(β), data, m3)
println("Cv = $m3 ± $s3")

Cv = 1.5295391962378664 ± 0.006430347477976632


## Autocorrelation

The simplest way to estimate autocorrelation is by changing the size of binning and estimating its errors by jackknife resampling.

In [11]:
binning = Iterators.partition(data, 2)
bin2 = mean.(binning)

5000-element Array{Array{Float64,1},1}:
 [-7.73421, -0.0111996] 
 [-7.60468, -0.00876931]
 [-7.7599, -0.011203]   
 [-7.67645, -0.00964333]
 [-7.57879, -0.00920894]
 [-7.66924, -0.00952568]
 [-7.76391, -0.0112659] 
 [-7.70879, -0.00526545]
 [-7.71375, -0.0134693] 
 [-7.68425, -0.0107764] 
 [-7.7144, -0.0118678]  
 [-7.69625, -0.0102179] 
 [-7.70269, -0.0118509] 
 ⋮                      
 [-7.71872, -0.0112845] 
 [-7.7139, -0.0118051]  
 [-7.72261, -0.00840472]
 [-7.65882, -0.00992718]
 [-7.67175, -0.0101091] 
 [-7.66054, -0.00896678]
 [-7.58177, -0.00762599]
 [-7.69802, -0.00869992]
 [-7.63575, -0.0100825] 
 [-7.68219, -0.0130865] 
 [-7.67022, -0.00972592]
 [-7.67996, -0.0124567] 

In [12]:
m4 = meanJ(TTCv, Cv(β), bin2)
s4 = stdmJ(TTCv, Cv(β), bin2, m4)
println("Cv = $m4 ± $s4")

Cv = 1.3066750164135335 ± 0.0048817582274606


The fact that the binsize does not affect the expectation value (or the errorbar) too much means that the autocorrelation length is about 1 step. Note that at low temperature the binsize strongly affects the expectation value, which means that the binsize must be taken to be large enough. Here I estimate the autocorrelation length by another method.

In [13]:
using StatsBase
StatsBase.autocor(first.(data))

41-element Array{Float64,1}:
  1.0                   
  0.0161154630920217    
  0.0016865836148915157 
 -0.004243363697991988  
  0.005083668849122723  
  0.015193291539698922  
 -0.0010670455271180055 
  0.004930228339101936  
 -0.009408070889752979  
  0.014811637146997908  
  0.0032928226280942957 
  0.00037346255897390224
 -0.002714883564312555  
  ⋮                     
 -0.0011090209306292489 
 -0.004735451673472965  
 -0.01977096431255157   
 -0.010740422219606163  
  0.010039901891214216  
  0.0018351614751997364 
  0.001386552020333793  
 -0.012728531021356883  
  0.0037777392141117275 
  0.0007988953543704431 
  0.008118742202624075  
  0.013352325602503079  

Rapid decay in the autocorrelation function means that the autocorrelation length is less than 1.

## Bootstrap method

~ under construction ~

**Exercise**: implement a Bootstrap method.